In [1]:
import sys
sys.path.append('../')

In [2]:
import itertools
import pandas as pd
import numpy as np
from tqdm.auto import tqdm, trange
from pathlib import Path

from models.train import test
from models.utils import ContagionDataset, set_seed, ConfusionMatrix
from sklearn.metrics import matthews_corrcoef, mean_squared_error
from models.results import ResultCollection

Using backend: pytorch


In [3]:
seed = 4444

metric_filter_1 = 'val_mcc'
metric_filter_2 = 'test_mcc'

data_dir_name = 'data'
save_path_name = 'saved_'
networks = ['sym_network', 'europe_network']

target = 'additional_stress'
dict_sets_lengths = {
    '75':(0.5, 0.25, 0.25),
    '40':(0.3, 0.1, 0.6),
    '10':(0.07, 0.03, 0.9),
}


Get all models that will be tested

In [7]:
# col = ResultCollection()
# paths = list(itertools.chain.from_iterable([[k for k in Path(n).glob(f"models*/{save_path_name}*") if k.is_dir()] for n in networks]))
network_paths = [[k for k in Path(n).glob(f"models*/{save_path_name}*") if k.is_dir()] for n in networks]

In [8]:
col_results = {}

for n,paths in zip(networks, network_paths):
    col = ResultCollection()
    col_results[n] = col

    for p in paths:
        name = p.name.split(save_path_name)[1]
        sets_type = p.parent.name.split('_')[2]
        sets_lengths = dict_sets_lengths[sets_type]
        data_dir = p.parent.parent.joinpath(data_dir_name)
        
        set_seed(seed)
        dataset_val = ContagionDataset(
            raw_dir=data_dir,
            drop_edges=0,
            sets_lengths=sets_lengths,
            target = target,
            add_self_loop=True,
        )

        r = test(
            dataset=dataset_val,
            save_path=str(p),
            n_runs=1,
            debug_mode=False,
            use_cpu=False,
            save=True,
            use_edge_weight=True,
        )

        result = col.add(r[2], f"{p.parent.name}_{name}")


  0%|          | 0/743 [00:00<?, ?it/s]

# separar networks

In [34]:
for p in paths:
    name = p.name.split(save_path_name)[1]
    sets_type = p.parent.name.split('_')[2]
    sets_lengths = dict_sets_lengths[sets_type]
    data_dir = p.parent.parent.joinpath(data_dir_name)
    
    set_seed(seed)
    dataset_val = ContagionDataset(
        raw_dir=data_dir,
        drop_edges=0,
        sets_lengths=sets_lengths,
        target = target,
        add_self_loop=True,
    )

    r = test(
        dataset=dataset_val,
        save_path=str(p),
        n_runs=1,
        debug_mode=False,
        use_cpu=False,
        save=True,
        use_edge_weight=True,
    )

    result = col.add(r[2], f"{p.parent.name}_{name}")

    break

100%|██████████| 743/743 [00:28<00:00, 26.20it/s]


In [35]:
result.data[0].keys()

dict_keys(['dict', 'train_cm', 'val_cm', 'test_cm'])

In [36]:
result.df_metrics_sort('test_mcc')

,train_optimizer_name,train_acc,val_acc,path_name,train_mcc,val_mcc,test_mcc,train_rmse,val_rmse,test_rmse,...,train_mae,val_mae,test_mae,train_rmse_perc,val_rmse_perc,test_rmse_perc,train_mae_perc,val_mae_perc,test_mae_perc,name
0,adamw,0.495238,0.400000,c:\Users\vibal\PycharmProjects\systemic-risk-p...,0.314632,0.261668,0.341909,1.133893,1.220200,1.087641,...,0.752381,0.866667,0.722222,0.280758,0.296383,0.267782,0.211190,0.222111,0.202422,models_clas_10_fnn
1,adamw,0.476190,0.400000,c:\Users\vibal\PycharmProjects\systemic-risk-p...,0.285387,0.261668,0.337861,1.167007,1.220200,1.086619,...,0.790476,0.866667,0.722963,0.287628,0.296383,0.267900,0.219000,0.222111,0.202926,models_clas_10_fnn
2,adamw,0.476190,0.400000,c:\Users\vibal\PycharmProjects\systemic-risk-p...,0.285387,0.261668,0.337861,1.167007,1.220200,1.086619,...,0.790476,0.866667,0.722963,0.287628,0.296383,0.267900,0.219000,0.222111,0.202926,models_clas_10_fnn
3,adamw,0.476190,0.400000,c:\Users\vibal\PycharmProjects\systemic-risk-p...,0.285387,0.261668,0.337861,1.167007,1.220200,1.086619,...,0.790476,0.866667,0.722963,0.287628,0.296383,0.267900,0.219000,0.222111,0.202926,models_clas_10_fnn
4,adamw,0.476190,0.400000,c:\Users\vibal\PycharmProjects\systemic-risk-p...,0.285387,0.261668,0.337861,1.167007,1.220200,1.086619,...,0.790476,0.866667,0.722963,0.287628,0.296383,0.267900,0.219000,0.222111,0.202926,models_clas_10_fnn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
738,adamw,0.247619,0.355555,c:\Users\vibal\PycharmProjects\systemic-risk-p...,0.069825,0.019556,-0.006653,1.877181,1.605546,1.866270,...,1.485714,1.200000,1.499259,0.487136,0.402159,0.475094,0.396714,0.312778,0.393437,models_clas_10_fnn
739,adamw,0.304762,0.200000,c:\Users\vibal\PycharmProjects\systemic-risk-p...,0.004183,0.002868,-0.020629,1.444200,1.273665,1.387777,...,1.114286,1.044444,1.115556,0.379870,0.315803,0.358489,0.303952,0.261667,0.295593,models_clas_10_fnn
740,adamw,0.276190,0.133333,c:\Users\vibal\PycharmProjects\systemic-risk-p...,-0.027391,-0.025851,-0.027899,1.869556,1.988858,1.836462,...,1.495238,1.688889,1.480741,0.475160,0.493804,0.463321,0.395667,0.425444,0.382844,models_clas_10_fnn
741,adamw,0.238095,0.088889,c:\Users\vibal\PycharmProjects\systemic-risk-p...,-0.097734,-0.121046,-0.063439,1.849067,2.033060,1.810463,...,1.514286,1.777778,1.483704,0.471842,0.509530,0.462657,0.395667,0.453222,0.386756,models_clas_10_fnn


In [37]:
result.data[0]['dict']['path_name']

WindowsPath('c:/Users/vibal/PycharmProjects/systemic-risk-predictor/notebooks/sym_network/models_clas_10/saved_fnn/0.09_4_[4_4]_4_ReLU()_None_0.0_0.1_adamw_max_val_mcc_0.0_False_False_0')

In [ ]:
result.save_best('test_mcc', '.',)